In [29]:
import sys
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from dataclasses import dataclass, field
from typing import List, Optional
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from glob import glob
import random
from tensorflow.keras.callbacks import Callback 
import json
from dataclasses import dataclass
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from concurrent.futures import ProcessPoolExecutor, as_completed
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
import torch
import torch.nn as nn
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [37]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
# 定义需要显示的特定面部关键点编号
selected_indices = [
    419, 290, 303, 242, 56, 155, 221, 226, 387, 362, 385, 310, 295, 340, 0, 37, 39, 40, 178, 146, 90, 72, 
    448, 380, 274, 398, 87, 98, 64, 324, 222, 1, 13, 22, 159, 145, 157, 89, 
    312, 462, 259, 63, 66, 112, 461, 463, 348, 62, 308, 119, 269, 78, 16, 65, 144, 163, 
    384, 229, 84, 321, 325, 466, 403, 182, 232, 219, 141, 249, 196, 320, 95, 
    304, 77, 272, 224, 239, 268, 316, 405, 86, 186, 
    63, 296, 334, 53, 195, 66, 107, 52, 65
]

# 定义数据类来存储每个关键点的坐标
@dataclass
class Landmark:
    x: float
    y: float
    z: float

# 定义 CustomResults 数据类
@dataclass
class CustomResults:
    face_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None
    pose_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None
    left_hand_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None
    right_hand_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None

# 提取并过滤面部关键点并生成 NormalizedLandmarkList
def create_filtered_face_landmarks(landmarks, indices):
    if not landmarks:
        return None
    filtered_landmarks = [landmarks[idx] for idx in indices]
    return landmark_pb2.NormalizedLandmarkList(landmark=filtered_landmarks)
# 使用标准的 MediaPipe 绘图方式来绘制关键点（无连接线）
def draw_landmarks(image, custom_results):
    h, w, _ = image.shape
    
    # 绘制面部关键点（不包含连接线）
    if custom_results.face_landmarks:
        for idx, landmark in zip(selected_indices, custom_results.face_landmarks.landmark):
            x, y = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)  
            cv2.putText(image, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)

    # 绘制姿势关键点
    if custom_results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, custom_results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(88, 22, 88), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(66, 22, 66), thickness=2, circle_radius=2)
        )

    # 绘制左手关键点
    if custom_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, custom_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(100, 22, 200), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(100, 22, 200), thickness=2, circle_radius=2)
        )

    # 绘制右手关键点
    if custom_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, custom_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 90, 0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 90, 0), thickness=2, circle_radius=2)
        )

def extract_keypoints(custom_results):
    face = np.array([[lm.x, lm.y, lm.z] for lm in custom_results.face_landmarks.landmark]).flatten() \
        if custom_results.face_landmarks else np.full(90 * 3, np.nan)
        
    pose = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in custom_results.pose_landmarks.landmark]).flatten() \
        if custom_results.pose_landmarks else np.full(33 * 4, np.nan)
        
    lh = np.array([[lm.x, lm.y, lm.z] for lm in custom_results.left_hand_landmarks.landmark]).flatten() \
        if custom_results.left_hand_landmarks else np.full(21 * 3, np.nan)
        
    rh = np.array([[lm.x, lm.y, lm.z] for lm in custom_results.right_hand_landmarks.landmark]).flatten() \
        if custom_results.right_hand_landmarks else np.full(21 * 3, np.nan)
    
    # 合并所有关键点
    keypoints = np.concatenate([face, pose, lh, rh])
    return keypoints


In [30]:


def ensure_sequence_shape(sequence):
    """
    确保输入的 sequence 形状为 (seq_len, n_keypoints, 3)。
    如果输入是 2D，将其重塑为 3D。
    """
    if sequence.dim() == 2 and sequence.shape[1] % 3 == 0:
        n_keypoints = sequence.shape[1] // 3
        sequence = sequence.view(sequence.shape[0], n_keypoints, 3)
    elif sequence.dim() != 3:
        raise ValueError(f"Expected sequence to have shape (seq_len, n_keypoints, 3), but got {sequence.shape}")
    return sequence


def compute_mean_std(X_train):
    """
    计算数据集 X_train 的全局均值和标准差。
    """
    all_keypoints = np.concatenate([sample for sample in X_train], axis=0)
    mean = np.nanmean(all_keypoints, axis=0)
    std = np.nanstd(all_keypoints, axis=0)
    return mean, std


def temporal_resample(sequence, target_length=105, min_scale=0.5, max_scale=1.5):
    sequence = ensure_sequence_shape(sequence)
    scale = np.random.uniform(min_scale, max_scale)
    new_length = int(sequence.shape[0] * scale)

    n_keypoints = sequence.shape[1]
    sequence = sequence.permute(1, 0, 2).contiguous().view(-1, sequence.shape[0])
    resampled_sequence = F.interpolate(sequence.unsqueeze(0), size=new_length, mode='linear', align_corners=False).squeeze(0)
    resampled_sequence = resampled_sequence.view(n_keypoints, new_length, 3).permute(1, 0, 2)
    final_sequence = F.interpolate(resampled_sequence.permute(1, 2, 0).contiguous().view(-1, resampled_sequence.shape[0]).unsqueeze(0),
                                   size=target_length, mode='linear', align_corners=False).squeeze(0)
    final_sequence = final_sequence.view(n_keypoints, target_length, 3).permute(1, 0, 2)
    return final_sequence


def windowed_cutmix(sequence1, sequence2, target_length=105):
    sequence1, sequence2 = ensure_sequence_shape(sequence1), ensure_sequence_shape(sequence2)
    cut_ratio = np.random.rand()
    cut_point1 = int(sequence1.shape[0] * cut_ratio)
    cut_point2 = int(sequence2.shape[0] * cut_ratio)
    mixed_sequence = torch.cat((sequence1[:cut_point1], sequence2[cut_point2:]), dim=0)
    mixed_sequence = mixed_sequence.permute(1, 0, 2).contiguous()
    mixed_sequence = mixed_sequence.view(-1, mixed_sequence.shape[1])
    final_sequence = F.interpolate(mixed_sequence.unsqueeze(0), size=target_length, mode='linear', align_corners=False).squeeze(0)
    final_sequence = final_sequence.view(sequence1.shape[1], target_length, 3).permute(1, 0, 2)
    return final_sequence


def temporal_shift(sequence, target_length=105, max_shift=10):
    sequence = ensure_sequence_shape(sequence)
    shift = np.random.randint(-max_shift, max_shift)
    shifted_sequence = torch.roll(sequence, shifts=shift, dims=0)
    seq_len = shifted_sequence.shape[0]
    if seq_len > target_length:
        shifted_sequence = F.interpolate(shifted_sequence.permute(1, 2, 0).unsqueeze(0), size=target_length, mode='linear', align_corners=False)
        shifted_sequence = shifted_sequence.squeeze(0).permute(2, 0, 1)
    elif seq_len < target_length:
        padding = torch.zeros((target_length - seq_len, shifted_sequence.shape[1], shifted_sequence.shape[2]), dtype=shifted_sequence.dtype)
        shifted_sequence = torch.cat((shifted_sequence, padding), dim=0)
    return shifted_sequence


def random_keypoint_dropout(sequence, num_points_to_drop=6, num_time_windows=3):
    sequence = ensure_sequence_shape(sequence)
    seq_len, num_keypoints, _ = sequence.shape
    for _ in range(num_time_windows):
        start = np.random.randint(0, seq_len)
        end = min(seq_len, start + np.random.randint(1, seq_len // num_time_windows))
        drop_indices = np.random.choice(num_keypoints, num_points_to_drop, replace=False)
        sequence[start:end, drop_indices, :] = 0
    return sequence


def spatial_mask(sequence, mask_prob=0.3, max_points=10):
    sequence = ensure_sequence_shape(sequence)
    if np.random.rand() < mask_prob:
        num_keypoints = sequence.shape[1]
        mask_points = np.random.choice(num_keypoints, max_points, replace=False)
        sequence[:, mask_points, :] = 0
    return sequence


def temporal_mask(sequence, mask_prob=0.3, max_mask_len=10):
    sequence = ensure_sequence_shape(sequence)
    if np.random.rand() < mask_prob:
        seq_len = sequence.shape[0]
        mask_len = np.random.randint(1, max_mask_len)
        start = np.random.randint(0, seq_len - mask_len)
        sequence[start:start + mask_len, :, :] = 0
    return sequence

def drop_face_or_pose(sequence, drop_face_prob=0.2, drop_pose_prob=0.2, face_indices=None, pose_indices=None):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    if np.random.rand() < drop_face_prob and face_indices is not None:
        sequence[:, face_indices, :] = 0  # 面部关键点置为 0
    if np.random.rand() < drop_pose_prob and pose_indices is not None:
        sequence[:, pose_indices, :] = 0  # 姿态关键点置为 0
    return sequence

def drop_hand_keypoints(sequence, drop_hand_prob=0.05, left_hand_indices=None, right_hand_indices=None):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    if np.random.rand() < drop_hand_prob:
        if left_hand_indices is not None:
            sequence[:, left_hand_indices, :] = 0  # 左手关键点置为 0
        if right_hand_indices is not None:
            sequence[:, right_hand_indices, :] = 0  # 右手关键点置为 0
    return sequence

def flip_keypoints(sequence, left_hand_indices, right_hand_indices):
    """
    仅对左右手的关键点进行翻转。

    Args:
        sequence (torch.Tensor): 形状为 (seq_len, n_keypoints, 3) 的关键点序列。
        left_hand_indices (list): 左手关键点索引。
        right_hand_indices (list): 右手关键点索引。
    
    Returns:
        torch.Tensor: 翻转后的关键点序列。
    """
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)

    # 翻转 X 坐标
    sequence[:, :, 0] = -sequence[:, :, 0]

    # 创建副本，用于交换左右手关键点
    flipped_sequence = sequence.clone()
    for l_idx, r_idx in zip(left_hand_indices, right_hand_indices):
        flipped_sequence[:, r_idx, :] = sequence[:, l_idx, :]  # 左手 → 右手
        flipped_sequence[:, l_idx, :] = sequence[:, r_idx, :]  # 右手 → 左手

    return flipped_sequence
def affine_transform(sequence, scale_range=(0.9, 1.1), translation_range=(-0.1, 0.1), rotation_range=(-10, 10)):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)

    # 随机缩放
    scale = np.random.uniform(*scale_range)
    sequence = sequence * scale

    # 随机平移
    translation = np.random.uniform(*translation_range, size=(1, sequence.shape[1], sequence.shape[2]))
    sequence = sequence + torch.tensor(translation, dtype=sequence.dtype, device=sequence.device)

    # 随机旋转（围绕 Z 轴旋转）
    angle = np.radians(np.random.uniform(*rotation_range))
    rotation_matrix = torch.tensor([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0, 0, 1]
    ], dtype=sequence.dtype, device=sequence.device)
    sequence = torch.matmul(sequence, rotation_matrix)
    return sequence



def apply_random_augmentations(sequence, augment_prob=0.5, target_length=105):
    """
    对序列数据应用增强操作，支持未展平的 3D 数据。
    sequence 形状: (seq_len, n_keypoints, feature_dim)
    """
    sequence = ensure_sequence_shape(sequence)  # 确保序列为 3D 形状 (seq_len, n_keypoints, 3)
    print(f"Initial sequence shape: {sequence.shape}")

    # 日志记录用于追踪应用的增强操作
    logs = []

    # 1. 时间增强
    if np.random.rand() < augment_prob:
        sequence = temporal_resample(sequence, target_length=target_length)
        logs.append("Applied temporal_resample")

    if np.random.rand() < augment_prob:
        sequence = temporal_shift(sequence)
        logs.append("Applied temporal_shift")

    # 2. 空间增强
    if np.random.rand() < augment_prob:
        sequence = affine_transform(sequence)
        logs.append("Applied affine_transform")

    # 3. 遮罩增强
    if np.random.rand() < augment_prob:
        sequence = drop_face_or_pose(sequence, face_indices=face_indices, pose_indices=pose_indices)
        logs.append("Applied drop_face_or_pose")
    if np.random.rand() < augment_prob:
        sequence = drop_hand_keypoints(sequence, left_hand_indices=left_hand_indices, right_hand_indices=right_hand_indices)
        logs.append("Applied drop_hand_keypoints")
    if np.random.rand() < augment_prob:
        sequence = spatial_mask(sequence)
        logs.append("Applied spatial_mask")
    if np.random.rand() < augment_prob:
        sequence = temporal_mask(sequence)
        logs.append("Applied temporal_mask")

    # 4. 翻转增强
    if np.random.rand() < augment_prob:
        sequence = flip_keypoints(sequence, left_hand_indices, right_hand_indices)
        logs.append("Applied flip_keypoints")

    # 打印日志
    if logs:
        print(" | ".join(logs))
    sequence = sequence.reshape(sequence.shape[0], -1)  # 展平为 (seq_len, n_keypoints * 3)


    print(f"Final sequence shape: {sequence.shape}")
    return sequence


DATA_PATH = "I:\\Ece496\\custom_data\\preprocess"
target_length = 105  # 设置目标长度
batch_size = 8
num_epochs = 50
learning_rate = 0.001

# 加载数据
X = np.load(os.path.join(DATA_PATH, "X.npy"), allow_pickle=True)
y = np.load(os.path.join(DATA_PATH, "y.npy"), allow_pickle=True)
X = np.nan_to_num(X, nan=0.0)
# 数据集划分
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 保存划分后的数据
np.save(os.path.join(DATA_PATH, "X_train.npy"), X_train)
np.save(os.path.join(DATA_PATH, "y_train.npy"), y_train)
np.save(os.path.join(DATA_PATH, "X_val.npy"), X_val)
np.save(os.path.join(DATA_PATH, "y_val.npy"), y_val)
np.save(os.path.join(DATA_PATH, "X_test.npy"), X_test)
np.save(os.path.join(DATA_PATH, "y_test.npy"), y_test)

# 定义自定义数据集类
class ASLDataset(Dataset):
    def __init__(self, X, y, global_mean, global_std, augment=None):
        self.X = X
        self.y = y
        self.global_mean = global_mean
        self.global_std = global_std
        self.augment = augment

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sequence = self.X[idx]
        label = self.y[idx]

        # 替换 NaN 为 0
        sequence = np.nan_to_num(sequence, nan=0.0)

        # 数据标准化
        sequence = (sequence - self.global_mean) / self.global_std
        sequence = torch.from_numpy(sequence).float()

        if self.augment:
            sequence = apply_random_augmentations(sequence)


        return sequence, torch.tensor(label, dtype=torch.long)
    
global_mean, global_std = compute_mean_std(X_train)
# 在3D形状的 (seq_len, n_keypoints, 3) 中，定义关键点索引范围
face_indices = list(range(0, 90))            # 面部关键点在 3D 中为 0~89
pose_indices = list(range(90, 90 + 33))      # 姿态关键点在 3D 中为 90~122
left_hand_indices = list(range(123, 123 + 21))  # 左手关键点在 3D 中为 123~143
right_hand_indices = list(range(144, 144 + 21)) # 右手关键点在 3D 中为 144~164global_mean, global_std = compute_mean_std(X_train)
train_dataset = ASLDataset(X_train, y_train, global_mean, global_std, augment=True)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)



In [31]:
for epoch in range(1):  # 单个 epoch 测试
    print(f"Epoch {epoch + 1}")
    for batch_idx, (sequences, labels) in enumerate(train_loader):
        print(f"Batch {batch_idx + 1}:")
        for sample_idx, sequence in enumerate(sequences):
            if sample_idx < 5:  # 每个批次最多打印前 5 个样本
                print(f"  Sample {sample_idx + 1} enhancements:")
                apply_random_augmentations(sequence)
            else:
                break  # 只打印 5 条样本
        break  # 只打印一个批次，确保代码运行后不输出过多信息
    break
for epoch in range(1):  # 单个 epoch 测试
    print(f"Epoch {epoch + 1}")
    for batch_idx, (sequences, labels) in enumerate(train_loader):
        print(f"Batch {batch_idx + 1}:")
        for sample_idx, sequence in enumerate(sequences):
            if sample_idx < 5:  # 每个批次最多打印前 5 个样本
                print(f"  Sample {sample_idx + 1} enhancements:")
                apply_random_augmentations(sequence)
            else:
                break  # 只打印 5 条样本
        break  # 只打印一个批次，确保代码运行后不输出过多信息
    break

Epoch 1
Initial sequence shape: torch.Size([105, 176, 3])
Applied temporal_shift | Applied affine_transform | Applied drop_hand_keypoints | Applied spatial_mask | Applied temporal_mask
Final sequence shape: torch.Size([105, 528])
Initial sequence shape: torch.Size([105, 176, 3])
Applied temporal_resample | Applied drop_face_or_pose | Applied spatial_mask | Applied temporal_mask
Final sequence shape: torch.Size([105, 528])
Initial sequence shape: torch.Size([105, 176, 3])
Applied temporal_resample | Applied temporal_shift | Applied affine_transform | Applied drop_face_or_pose | Applied temporal_mask
Final sequence shape: torch.Size([105, 528])
Initial sequence shape: torch.Size([105, 176, 3])
Applied temporal_shift | Applied affine_transform | Applied drop_hand_keypoints | Applied temporal_mask | Applied flip_keypoints
Final sequence shape: torch.Size([105, 528])
Initial sequence shape: torch.Size([105, 176, 3])
Applied temporal_shift | Applied affine_transform | Applied drop_face_or_po

In [32]:
words = ['hello',  'I or me', 'father', 'mother','see u later']
label_map = {label: num for num, label in enumerate(words)}

In [35]:
model = Sequential()
# 使用 'tanh' 激活函数以便利用 cuDNN 优化
model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(105, 528)))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(len(words), activation='softmax'))

# 编译模型，使用 'accuracy' 作为度量指标
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [36]:
log_dir_with_velocity = os.path.join('Logs', 'with_velocity')
tb_callback_with_velocity = TensorBoard(log_dir=log_dir_with_velocity)

# 定义 EarlyStopping 回调：当验证损失在 5 个 epoch 后没有改善时停止训练
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# 定义 ReduceLROnPlateau 回调：如果验证损失连续 3 个 epoch 无改善则降低学习率
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# 将回调函数放入列表中
callbacks = [tb_callback_with_velocity , early_stopping, reduce_lr]
class TestCallback(Callback):
    def __init__(self, model, X_test, y_test):
        super(TestCallback, self).__init__()
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
    

    def on_epoch_end(self, epoch, logs=None):
        # 这里可以添加评估测试数据集的代码
        test_loss, test_acc = self.model.evaluate(self.X_test, self.y_test, verbose=0)
        print(f'   Test loss: {test_loss}, Test accuracy: {test_acc}')
        # 可以根据需要添加更多的自定义行为

test_callback = TestCallback(model, X_test, y_test)

history = model.fit(
    X_train, 
    y_train, 
    epochs=100, 
    validation_data=(X_val, y_val), 
    callbacks=[tb_callback_with_velocity, test_callback]  # 添加自定义回调
)

Epoch 1/100
13/13 [==============================] - 6s 135ms/step - loss: 2.0802 - accuracy: 0.2157 - val_loss: 1.6203 - val_accuracy: 0.2353
Epoch 2/100
13/13 [==============================] - 1s 49ms/step - loss: 1.8370 - accuracy: 0.2206 - val_loss: 1.6262 - val_accuracy: 0.1373
Epoch 3/100
13/13 [==============================] - 1s 50ms/step - loss: 1.7176 - accuracy: 0.2230 - val_loss: 1.6096 - val_accuracy: 0.1373
Epoch 4/100
13/13 [==============================] - 1s 51ms/step - loss: 1.5992 - accuracy: 0.3162 - val_loss: 1.6380 - val_accuracy: 0.1569
Epoch 5/100
13/13 [==============================] - 1s 46ms/step - loss: 1.6302 - accuracy: 0.2623 - val_loss: 1.6350 - val_accuracy: 0.1373
Epoch 6/100
13/13 [==============================] - 1s 52ms/step - loss: 1.5553 - accuracy: 0.2917 - val_loss: 1.5831 - val_accuracy: 0.3137
Epoch 7/100
13/13 [==============================] - 1s 46ms/step - loss: 1.5379 - accuracy: 0.3431 - val_loss: 1.5672 - val_accuracy: 0.2941
Epoch

In [38]:
TEST_VIDEO_FOLDER = "I:\\Ece496\\custom_data\\realtime_test"
mp_holistic = mp.solutions.holistic
video_files = glob(os.path.join(TEST_VIDEO_FOLDER, "*.mp4"))

video_files = glob(os.path.join(TEST_VIDEO_FOLDER, "*.mp4"))

# 处理每个视频
for video_file in video_files:
    video_name = os.path.basename(video_file)
    print(f"Processing video: {video_name}")

    cap = cv2.VideoCapture(video_file)
    sequence = []  # 存储每帧的关键点
    predicted_labels = []  # 存储每帧的预测标签

    # 使用 MediaPipe 处理视频
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # 转换颜色空间并处理帧
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(frame_rgb)

            # 提取关键点并累加到序列中
            keypoints = extract_keypoints(results)
            if keypoints is not None:
                sequence.append(keypoints)

            # 检查是否达到 105 帧
            if len(sequence) == 105:
                prediction = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
                label = np.argmax(prediction)
                predicted_labels.append(label)

                # 清空 sequence 以便下一段 105 帧
                sequence = []

    cap.release()

    # 统计出现最多的标签作为最终预测结果
    if predicted_labels:
        final_prediction = max(set(predicted_labels), key=predicted_labels.count)
        print(f"Video '{video_name}' processed. Final predicted label: {final_prediction} (Action: {actions[final_prediction]})\n")
    else:
        print(f"Video '{video_name}' processed. No prediction available.\n")

Processing video: I or me_1_alice.mp4


ValueError: in user code:

    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\input_